<a href="https://colab.research.google.com/github/junieberry/NLP-withPyTorch/blob/main/05_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# annoy 패키지를 설치합니다.
!pip install annoy

     |████████████████████████████████| 646 kB 5.5 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391603 sha256=ac01381fbba6aa63421bf6e2957e9d92be49f4ff46f2a54d6e4d1570bd088896
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:

import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np


## 5.1 단어 임베딩을 배우는 이유

**단어 벡터 표현을 만드는 전통적인 방법**

1. 원-핫 표현

  벡터의 길이는 어휘사전의 크기와 같고, 값은 1 혹은 0이다.

2. 카운트 기반 표현

  벡터의 길이는 어휘사전의 크기와 같지만 값은 단어의 빈도에 상응한다.

  경험적으로 얻어진다.


**밀집 벡터의 장점**

1. 계산의 효율성
2. 통계적 장점
3. 고차원 입력은.. *차원의저주*


### 5.1.2 단어 임베딩 학습 방법

**단어의 통계적, 언어적 속성을 감지하기 위한 보조 작업**

1. 단어 시퀀스가 주어지면 다음 단어를 예측
2. 앞 뒤 단어 시퀀스가 주어지면 가운데 단어를 예측 (CBOW)
3. 단어가 주어지면 위치와 상관 없는 단어 예측 (Skipgram)

e.g GloVe, CBOW (continuous Bag of Word), Skipgram

### 5.1.3 사전 훈련된 단어 임베딩

**임베딩 로드**

임베딩 포맷 = (임베딩 단어 / 임베딩 벡터 표현)

`PreTrainedEmbedding` 클래스를 구현해보자!

In [8]:
class PreTrainedEmbedding(object):

  def __init__(self, word_to_index, word_vectors):

    self.word_to_index = word_to_index
    self.word_vectors = word_vectors
    ##
    self.index_to_word = {v: k for k, v in self.word_to_index.items()}

    ## 인덱스 만들기
    self.index = AnnoyIndex(len(word_vectors[0]), metric="euclidean")
    for _, i in self.word_to_index.items():
      self.index.add_item(i, self.word_vectors[i])
    self.index.build(50) ## ????
  

  # 주어진 파일으로 PretrainedEmbedding 인스턴스 반환
  ## embedding_file = 파일 위치
  ## return == PretrainedEmbedding 인스턴스
  @classmethod
  def from_embeddings_file(cls, embedding_file):
    
    word_to_index = {}
    word_vectors = []

    with open(embedding_file) as fp:
      for line in fp.readlines():
        line = line.split(" ")
        word = line[0]
        vec = np.array([float(x) for x in line[1:]])

        word_to_index[word] = len(word_to_index)
        word_vectors.append(vec)

    return cls(word_to_index, word_vectors)
  
  # 주어진 단어의 임베딩 표현 반환
  ## word = 단어
  ## return == 임베딩 (numpy.ndarray)
  def get_embedding(self, word):
    return self.word_vectors[self.word_to_index[word]]
  
  # 주어진 벡터의 n개의 최근접 이웃 반환 (Annoy가 해줌)
  ## vector = 주어진 벡터 (np.ndarray)
  ## n = 반횐될 이웃의 개수 (int)
  ## return == 주어진 벡터와 가까운 단어들
  def get_closest_to_vector(self, vector, n=1):
    nn_indices = self.index.get_nns_by_vector(vector, n)
    return [self.index_to_word[neighbor] for neighbor in nn_indices]
  
  # 단어의 유추 관계
  ## word1 : word2 = word2 : word4
  ## word4 출력

  def compute_and_print_analogy(self, word1, word2, word3):

    vec1 = self.get_embedding(word1)
    vec2 = self.get_embedding(word2)
    vec3 = self.get_embedding(word3)

    spatial_relationship = vec2 - vec1
    vec4 =vec3 + spatial_relationship

    closest_words = self.get_closest_to_vector(vec4, n=4)
    existing_words = set([word1, word2, word3])
    closest_words = [word for word in closest_words if word not in existing_words] 

    if len(closest_words) == 0:
      print("계산된 벡터와 가장 가까운 이웃을 찾을 수 없습니다!")
      return
        
    for word4 in closest_words:
      print("{} : {} :: {} : {}".format(word1, word2, word3, word4))


In [3]:
# GloVe 데이터를 다운로드합니다.
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!mkdir -p data/glove
!mv glove.6B.100d.txt data/glove

--2021-09-22 10:43:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-22 10:43:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-22 10:43:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [10]:
embeddings = PreTrainedEmbedding.from_embeddings_file('data/glove/glove.6B.100d.txt')

In [11]:
## 성별 명사와 대명사의 관계
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : her


In [12]:
## 언어 규칙과 문화 편견
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician
man : doctor :: woman : doctors


## 5.2 CBOW 임베딩 학습하기



## 5.3 문서 분류에 사전 훈련된 임베딩을 사용한 전이 학습

## 5.4 요약

단어 임베딩의 편향 제거, 문맥 모델링, 다의성 등의 주제가 있다.

